In [4]:
import time
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

from tqdm import tqdm

import asyncio
import aiohttp

In [5]:
page_min = 1
page_max = 2858
# page_max = 2858
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'
}

final_dict = {
    'title': [],
    'out_year': [],
    'genres': [],
    'country': [],
    'duration': [],
    'description': [],
    'url_movie': [],
    'image_movie': [],
    'rating_votes': [],
    'film_rating_kp': [],
    'film_rating_imdb': []
}


In [6]:
for page in range(page_min, page_max+1):
    start_time = time.time()
    url = f'https://lux.kinogo.biz/page/{page}/'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    # 0. Найдём все блоки shortstory
    blocks = soup.find_all("div", class_="shortstory")

    # 1. Вытаскиваем все h2 из этих блоков, названия фильмов
    final_dict['title'].extend([block.find("h2").get_text(strip=True) if block.find("h2") else None for block in blocks])

    # 2. Вытаскиваем год выхода фильма
    final_dict['out_year'].extend([block.find("div", class_="shortstory__info-wrapper").find("a").get_text(strip=True) if block.find("div", class_="shortstory__info-wrapper").find("a") else None for block in blocks])

    # 3. Вытаскиваем URL фильма
    final_dict['url_movie'].extend([block.find("a").get("href") if block.find("a") else None for block in blocks])

    # 4. Вытаскиваем URL постера
    final_dict['image_movie'].extend(['https://lux.kinogo.biz' + block.find("div", class_="shortstory__poster").find("img").get('data-src') if block.find("div", class_="shortstory__poster").find("img") else None for block in blocks])
    
    # 5. Вытаскиваем жанры фильма
    for block in blocks:
        genre_title_tag = block.find('b', string='Жанр:')
        if genre_title_tag:
            genre_tags = genre_title_tag.find_next_siblings('a')
            movie_genres = [item.get_text() for item in genre_tags if item.get_text() not in ['Фильмы', 'Новинки']]
            final_dict['genres'].append(movie_genres)
        else:
            final_dict['genres'].append(None)
    
    # 6. Вытаскиваем страну производства фильма
    for block in blocks:
        country_title_tag = block.find('b', string='Страна:')
        if country_title_tag:
            country_tags = country_title_tag.find_next_siblings('a')
            movie_country = [item.get_text() for item in country_tags]
            final_dict['country'].append(movie_country)
        else:
            final_dict['country'].append(None)

    # 7. Вытаскиваем продолжительность фильма
    for block in blocks:
        duration_title_tag = block.find('b', string='Продолжительность:')
        if duration_title_tag:
            final_dict['duration'].append(duration_title_tag.next_sibling.strip())
        else:
            final_dict['duration'].append(None)
    
    # 8. Вытаскиваем описание фильма
    for block in blocks:
        description_title_tag = block.find("div", class_="excerpt")
        if description_title_tag:
            final_dict['description'].append(description_title_tag.get_text(strip=True))
        else:
            final_dict['description'].append(None)

    # 9. Вытаскиваем рейтинг фильма самого сайта 
    for block in blocks:
        rating_tag = block.find("div", class_="rating__votes")
        if rating_tag:
            final_dict['rating_votes'].append(rating_tag.get_text(strip=True))
        else:
            final_dict['rating_votes'].append(None)

    # 10. Вытаскиваем рейтинг фильма КИНО-ПОИСК
    for block in blocks:
        rating_kp_tag = block.find("span", class_="kp")
        if rating_kp_tag:
            final_dict['film_rating_kp'].append(rating_kp_tag.get_text(strip=True))
        else:
            final_dict['film_rating_kp'].append(None)

    # 11. Вытаскиваем рейтинг фильма IMDB
    for block in blocks:
        rating_imdb_tag = block.find("span", class_="imdb")
        if rating_imdb_tag:
            final_dict['film_rating_imdb'].append(rating_imdb_tag.get_text(strip=True))
        else:
            final_dict['film_rating_imdb'].append(None)

    end_time = time.time() - start_time
    print(f'Парсинг страницы {page} занял {end_time:.2f} сек')

    if page % 100 == 0:
        current_df = pd.DataFrame(final_dict)
        current_df.to_csv('movies_dataset.csv', index=False)
    if page == page_max:
        current_df = pd.DataFrame(final_dict)
        current_df.to_csv('movies_dataset.csv', index=False)

    time.sleep(np.random.rand())

Парсинг страницы 1 занял 1.78 сек
Парсинг страницы 2 занял 0.73 сек
Парсинг страницы 3 занял 0.83 сек
Парсинг страницы 4 занял 0.83 сек
Парсинг страницы 5 занял 0.87 сек
Парсинг страницы 6 занял 0.78 сек
Парсинг страницы 7 занял 0.75 сек
Парсинг страницы 8 занял 0.71 сек
Парсинг страницы 9 занял 0.75 сек
Парсинг страницы 10 занял 0.74 сек
Парсинг страницы 11 занял 0.82 сек
Парсинг страницы 12 занял 0.75 сек
Парсинг страницы 13 занял 0.74 сек
Парсинг страницы 14 занял 0.79 сек
Парсинг страницы 15 занял 0.76 сек
Парсинг страницы 16 занял 0.94 сек
Парсинг страницы 17 занял 0.74 сек
Парсинг страницы 18 занял 0.76 сек
Парсинг страницы 19 занял -0.28 сек
Парсинг страницы 20 занял 0.74 сек
Парсинг страницы 21 занял 0.74 сек
Парсинг страницы 22 занял 0.77 сек
Парсинг страницы 23 занял 0.76 сек
Парсинг страницы 24 занял 0.78 сек
Парсинг страницы 25 занял 0.77 сек
Парсинг страницы 26 занял 0.82 сек
Парсинг страницы 27 занял 0.78 сек
Парсинг страницы 28 занял 0.81 сек
Парсинг страницы 29 занял 0.

In [23]:
current_df.shape

(4000, 11)